<a href="https://colab.research.google.com/github/Polohodoro/DKU_Team12_AI-_for_Security/blob/main/%EC%A0%84%EC%B2%98%EB%A6%AC_%EC%BD%94%EB%93%9C(%EC%88%98%EC%A0%95)_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/misbah4064/human-pose-estimation-opencv.git

fatal: destination path 'human-pose-estimation-opencv' already exists and is not an empty directory.


In [ ]:
%cd human-pose-estimation-opencv

/content/human-pose-estimation-opencv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2 as cv
import numpy as np
from google.colab.patches import cv2_imshow

def cal_angle(point1, point2): #  단위벡터 구하기
    diff = np.array(point2) - np.array(point1)
    vector = np.linalg.norm(diff)

    if vector != 0:
        vector = diff / vector
    else:
        vector = np.zeros_like(diff)
    return vector[0],vector[1]

In [ ]:
import os
def get_video_files(directory_path):
    video_files = [f for f in os.listdir(directory_path) if f.endswith(('.mp4'))]
    video_files.sort()  # 파일을 정렬하려면 사용
    return video_files

# Google Drive 내의 디렉토리 경로 설정
path = "/content/drive/MyDrive/영상폴더/구매"

# 비디오 파일 목록 가져오기
video_files = get_video_files(path)

In [ ]:
import cv2 as cv
import numpy as np
from google.colab.patches import cv2_imshow

BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

# 내적값을 구할 관절 모음
# 필요없다고 판단한 얼굴부분 좌표는 제외시켰습니다.
POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"] ]

width = 368
height = 368
inWidth = width
inHeight = height

net = cv.dnn.readNetFromTensorflow("graph_opt.pb")
thr = 0.2

def poseDetector(frame):
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]

    net.setInput(cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :19, :, :]  # MobileNet output [1, 57, -1, -1], we only need the first 19 elements

    assert(len(BODY_PARTS) == out.shape[1])

    points = []
    for i in range(len(BODY_PARTS)):
        heatMap = out[0, i, :, :]
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]
        points.append((int(x), int(y)) if conf > thr else None)

    angles = []
    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]
        if points[idFrom] and points[idTo]:
            p_x,p_y = cal_angle(points[idFrom], points[idTo])
            joint_angles = {f'{partFrom}-{partTo}-x': p_x ,f'{partFrom}-{partTo}-y': p_y }
            angles.append(joint_angles)
        else:
            angles.append({f'{partFrom}-{partTo}-x': 0.0,f'{partFrom}-{partTo}-y':0.0})

    t, _ = net.getPerfProfile()

    # 관절 좌표 반환
    return frame, angles

일반 구매 데이터

In [ ]:
import cv2
import pandas as pd
import os

for file_name in video_files  :
 print(file_name)
 if not(f"{file_name}_1.pkl" in os.listdir("/content/drive/MyDrive/영상폴더/구매(수정)")):
  cap = cv2.VideoCapture(f"{path}/{file_name}")
  ret, frame = cap.read()
  frame_height, frame_width, _ = frame.shape
  out = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
  print("Processing Video...")
  coors = []
  angles = []
  while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
      out.release()
      break

    output,angle = poseDetector(frame)
    an = {}
    for item in angle:
      an.update(item)
    angles.append(an)
  df = pd.DataFrame(angles)
  df = df.fillna(0.0)
  total_rows = len(df)
  half_rows = total_rows // 2
  df1 = df.head(half_rows)
  df2 = df.tail(half_rows)

  df1.to_pickle(f"/content/drive/MyDrive/영상폴더/구매(수정)/{file_name}_1.pkl")
  df2.to_pickle(f"/content/drive/MyDrive/영상폴더/구매(수정)/{file_name}_2.pkl")
 else:
  print("pass")
print("Done")

C_2_2_57_BU_DYB_10-20_13-59-11_CA_RGB_DF1_M4_M4.mp4
pass
C_2_2_57_BU_DYB_10-20_13-59-11_CB_RGB_DF1_M4_M4.mp4
pass
C_2_2_57_BU_DYB_10-20_13-59-11_CC_RGB_DF1_M4_M4.mp4
pass
C_2_2_57_BU_DYB_10-20_13-59-11_CD_RGB_DF1_M4_M4.mp4
pass
C_2_2_57_BU_SMA_09-16_10-05-32_CC_RGB_DF1_F3_F3.mp4
pass
C_2_2_57_BU_SMA_09-16_10-05-33_CA_RGB_DF1_F3_F3.mp4
pass
C_2_2_57_BU_SMA_09-16_10-05-33_CB_RGB_DF1_F3_F3.mp4
pass
C_2_2_57_BU_SMB_10-15_15-56-14_CA_RGB_DF1_F3_F3.mp4
pass
C_2_2_57_BU_SMB_10-15_15-56-14_CB_RGB_DF1_F3_F3.mp4
pass
C_2_2_57_BU_SMB_10-15_15-56-14_CC_RGB_DF1_F3_F3.mp4
pass
C_2_2_57_BU_SMB_10-15_15-56-14_CD_RGB_DF1_F3_F3.mp4
pass
C_2_2_57_BU_SYA_09-15_10-48-45_CA_RGB_DF1_F3_F3.mp4
pass
C_2_2_57_BU_SYA_09-15_10-48-45_CB_RGB_DF1_F3_F3.mp4
pass
C_2_2_57_BU_SYA_09-15_10-48-45_CC_RGB_DF1_F3_F3.mp4
pass
C_2_2_57_BU_SYB_10-15_17-33-01_CA_RGB_DF1_F3_F3.mp4
pass
C_2_2_57_BU_SYB_10-15_17-33-01_CB_RGB_DF1_F3_F3.mp4
pass
C_2_2_57_BU_SYB_10-15_17-33-01_CC_RGB_DF1_F3_F3.mp4
pass
C_2_2_57_BU_SYB_10-15_17-33-01_

절도 데이터 (구매데이터 추출과 동일함)

In [ ]:
import os
def get_video_files(directory_path):
    video_files = [f for f in os.listdir(directory_path) if f.endswith(('.mp4'))]
    video_files.sort()  # 파일을 정렬하려면 사용
    return video_files

# Google Drive 내의 디렉토리 경로 설정
path = "/content/drive/MyDrive/영상폴더/절도"

# 비디오 파일 목록 가져오기
video_files = get_video_files(path)

In [ ]:
import cv2
import pandas as pd

for file_name in video_files  :
 print(file_name)
 if not(f"{file_name}.pkl" in os.listdir("/content/drive/MyDrive/영상폴더/절도(수정)")):
  cap = cv2.VideoCapture(f"{path}/{file_name}")
  ret, frame = cap.read()
  frame_height, frame_width, _ = frame.shape
  out = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
  print("Processing Video...")
  coors = []
  angles = []
  while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
      out.release()
      break

    output,angle = poseDetector(frame)
    an = {}
    for item in angle:
      an.update(item)
    angles.append(an)
  df = pd.DataFrame(angles)
  df = df.fillna(0.0)
  df.to_pickle(f"/content/drive/MyDrive/영상폴더/절도(수정)/{file_name}.pkl")
 else:
  print("pass")
print("Done")

C_3_12_10_BU_DYA_07-27_13-01-22_CA_RGB_DF2_F2.mp4
Processing Video...
C_3_12_10_BU_DYA_07-27_13-01-25_CB_RGB_DF2_F2.mp4
Processing Video...
C_3_12_10_BU_DYA_07-27_13-01-25_CC_RGB_DF2_F2.mp4
Processing Video...
C_3_12_10_BU_DYA_08-10_14-54-42_CD_RGB_DF2_M2.mp4
Processing Video...
C_3_12_10_BU_DYA_08-10_14-54-47_CE_RGB_DF2_M2.mp4
Processing Video...
C_3_12_10_BU_DYA_08-10_14-54-47_CF_RGB_DF2_M2.mp4
Processing Video...


KeyboardInterrupt: ignored